# import pvlib library

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../')) #append the parent directory to the system path
sys.path.append(os.path.abspath('../pvlib')) #append the parent directory to the system path
import pvlib as pvlib
import pandas as pd 
import datetime

# Import TMY data

## Use Sandia standard data 

In [2]:

fname='../../pvlib/data/723650TYA.csv' #Use absolute path if the file is not in the local directory
TMY, meta = pvlib.tmy.readtmy3(filename=fname)

In [3]:
# meta['longitude']=-79.2
# meta['latitude']=43.42
# meta['TZ']=-5
meta['SurfTilt']=30
meta['SurfAz']=0
meta['Albedo']=0.2

location = pvlib.location.Location(latitude=43.42, longitude=-79.2, tz='US/Mountain', altitude=100,
                 name=meta['Name'])

print(location)

"ALBUQUERQUE INTL ARPT [ISIS]": latitude=43.42, longitude=-79.2, tz=US/Mountain, altitude=100


In [4]:
print meta['State']
# print meta['longitude']
print(location)

NM
"ALBUQUERQUE INTL ARPT [ISIS]": latitude=43.42, longitude=-79.2, tz=US/Mountain, altitude=100


## Define date range of interest for graphing

In [5]:
month=3
day=26
hour=0
periods=24
freq='H'
rng=pd.date_range(datetime.datetime(year=min(TMY.index.year),month=month,day=day,hour=hour),periods=periods,freq=freq)
rng                

<class 'pandas.tseries.index.DatetimeIndex'>
[1979-03-26 00:00:00, ..., 1979-03-26 23:00:00]
Length: 24, Freq: H, Timezone: None

# Get solar angles

In [6]:
# import pvlib.solarposition
# reload(pvlib.solarposition)
# print(meta, type(meta))
#Using Ephemeris Calculations
# TMY['SunAz'],TMY['SunEl'],TMY['ApparentSunEl'],TMY['SolarTime'], TMY['SunZen'] = \
#      pvlib.solarposition.ephemeris(time=TMY.index,location=location)
df_ephemeris= pvlib.solarposition.ephemeris(time=TMY.index,location=location)
# print(df_ephemeris)    
TMY['SunAz'] = df_ephemeris['azimuth']
TMY['SunEl'] = df_ephemeris['elevation']
TMY['ApparentSunEl'] =  df_ephemeris['apparent_elevation']
TMY['SolarTime'] = df_ephemeris['solar_time']
TMY['SunZen'] =  df_ephemeris['zenith']
    
#Using NRELS SPA Calculations
# import pvl_spa
# reload (pvl_spa)
# TMY['SunAz_spa'],TMY['SunEl_spa'],TMY['SunZen_spa']=\
#     pvl_spa.pvl_spa(Time=TMY.index,Location=meta)
print(TMY.index.to_datetime(),type(TMY.index.to_datetime()))

dfout = pvlib.spa.solar_position(unixtime=TMY.index.to_pydatetime(), lat=location.latitude, lon=location.longitude, 
                    elev=location.altitude, 
                    pressure=101325, temp=12, delta_t=67.0, atmos_refract=0.5667)
print(dfout)



(<class 'pandas.tseries.index.DatetimeIndex'>
[1981-01-01 01:00:00-07:00, ..., 2000-01-01 00:00:00-07:00]
Length: 8760, Freq: None, Timezone: pytz.FixedOffset(-420), <class 'pandas.tseries.index.DatetimeIndex'>)


TypeError: unsupported operand type(s) for *: 'datetime.datetime' and 'float'

In [7]:
clf()
plot(TMY.index,TMY.SunAz)
plot(TMY.index,TMY.SunAz_spa)
plot(TMY.index,TMY.SunEl,label='eph')
plot(TMY.index,TMY.SunEl_spa,label='spa')
legend()

NameError: name 'clf' is not defined

# Calculate Extraterrestrial Irradiaion and AirMass

In [8]:

TMY['HExtra']=pvlib.pvl_extraradiation(doy=TMY.index.dayofyear)

TMY['AM']=pvlib.pvl_relativeairmass(z=TMY.SunZen)

AttributeError: 'module' object has no attribute 'pvl_extraradiation'

# Generate Clear Sky and DNI

In [9]:

DFOut=pvlib.pvl_disc(Time=TMY.index,GHI=TMY.GHI, SunZen=TMY.SunZen)

TMY['DNI_gen_DISC']=DFOut['DNI_gen_DISC']
TMY['Kt_gen_DISC']=DFOut['Kt_gen_DISC']
TMY['AM']=DFOut['AM']
TMY['Ztemp']=DFOut['Ztemp']


AttributeError: 'module' object has no attribute 'pvl_disc'

# Plane Transformation

In [11]:

TMY['In_Plane_SkyDiffuse']=pvlib.irradiance.perez(SurfTilt=meta['SurfTilt'],
                                            SurfAz=meta['SurfAz'],
                                            DHI=TMY.DHI,
                                            DNI=TMY.DNI,
                                            HExtra=TMY.HExtra,
                                            SunZen=TMY.SunZen,
                                            SunAz=TMY.SunAz,
                                            AM=TMY.AM)


AttributeError: 'DataFrame' object has no attribute 'HExtra'

# Ground Diffuse reflection

In [ ]:

TMY['GR']=pvlib.pvl_grounddiffuse(GHI=TMY.GHI,Albedo=meta['Albedo'],SurfTilt=meta['SurfTilt'])

# Get AOI

In [ ]:

TMY['AOI']=pvlib.pvl_getaoi(SunAz=TMY.SunAz,SunZen=TMY.SunZen,SurfTilt=meta['SurfTilt'],SurfAz=meta['SurfAz'])

# Calculate Global in-plane

In [ ]:

TMY['E'],TMY['Eb'],TMY['EDiff']=pvlib.pvl_globalinplane(AOI=TMY.AOI,
                                DNI=TMY.DNI,
                                In_Plane_SkyDiffuse=TMY.In_Plane_SkyDiffuse,
                                GR=TMY.GR,
                                SurfTilt=meta['SurfTilt'],
                                SurfAz=meta['SurfAz'])



# Calculate Cell Temperature

In [ ]:

TMY['Tcell'],TMY['Tmodule']=pvlib.pvl_sapmcelltemp(E=TMY.E,
                            Wspd=TMY.Wspd,
                            Tamb=TMY.DryBulb)





# Import module coefficients

In [ ]:

moddb=pvlib.pvl_retreiveSAM(name='SandiaMod')
module=moddb.Canadian_Solar_CS5P_220M___2009_
module

#  import inverter coefficients

In [ ]:
Invdb=pvlib.pvl_retreiveSAM(name='SandiaInverter')
inverter=Invdb.Advanced_Energy__Solaron_333_3159000_105_480V__CEC_2008_
inverter

# Sandia Model

In [ ]:

DFOut=pvlib.pvl_sapm(Eb=TMY['Eb'],
                    Ediff=TMY['EDiff'],
                    Tcell=TMY['Tcell'],
                    AM=TMY['AM'],
                    AOI=TMY['AOI'],
                    Module=module)

TMY['Imp']=DFOut['Imp']
TMY['Voc']=DFOut['Voc']
TMY['Vmp']=DFOut['Vmp']
TMY['Pmp']=DFOut['Pmp']
TMY['Ix']=DFOut['Ix']
TMY['Ixx']=DFOut['Ixx']

# Single Diode Model

In [ ]:

moddb=pvlib.pvl_retreiveSAM(name='CECMod')
module=moddb.Canadian_Solar_CS5P_220P

IL,I0,Rs,Rsh,nNsVth=pvlib.pvl_calcparams_desoto(S=TMY.GHI,
                                               Tcell=TMY.DryBulb,
                                               alpha_isc=.003,
                                               ModuleParameters=module,
                                               EgRef=1.121,
                                               dEgdT= -0.0002677)


DFout= pvlib.pvl_singlediode(Module=module,
                               IL=IL,
                               I0=I0,
                               Rs=Rs,
                               Rsh=Rsh,
                               nNsVth=nNsVth)


TMY['sd_Imp']=DFOut['Imp']
TMY['sd_Voc']=DFOut['Voc']
TMY['sd_Vmp']=DFOut['Vmp']
TMY['sd_Pmp']=DFOut['Pmp']
TMY['sd_Ix']=DFOut['Ix']
TMY['sd_Ixx']=DFOut['Ixx']           

# Inverter Model

In [ ]:

TMY['ACPower']=pvlib.pvl_snlinverter(Vmp=TMY.Vmp,Pmp=TMY.Pmp,Inverter=inverter)



## Python and [module versions, and dates](http://nbviewer.ipython.org/github/jrjohansson/scientific-python-lectures/blob/master/Lecture-0-Scientific-Computing-with-Python.ipynb)

In [ ]:
%load_ext version_information
%version_information numpy, scipy, matplotlib, pandas, pvlib